# Aggregate Functions and Group Statistics
* Contact: Lachlan Deer, [econgit] @ldeer, [github/twitter] @lachlandeer

So far our work with Pandas as been purely about simple transformations of each row or data; whether selecting them or creating new columns out of old ones. This is rarely enough - we typically want some degree of summarization of the data; either of an entire column; or on parts of (groups) of the column in question.

This is what we will go after here:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('out_data/state_labour_statistics.csv')

## Simple Aggregation

Like NumPy, Pandas has some of the standard data aggregations:

In [4]:
rng = np.random.RandomState(1234567890)
series = pd.Series(rng.rand(5))
series



0    0.618795
1    0.591624
2    0.888684
3    0.891655
4    0.457567
dtype: float64

In [5]:
series.sum()

3.4483242705353976

In [6]:
series.mean(), series.median(), series.max()

(0.6896648541070796, 0.61879477158568, 0.89165480011560816)

For a data-frame; these aggregations return results for each column:

In [7]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.778188,0.400999
1,0.267064,0.705406
2,0.996106,0.405186
3,0.540095,0.949991
4,0.537522,0.030754


In [10]:
df.mean()

A    0.623795
B    0.498467
dtype: float64

if you want the aggregation across a row, you can specify thats what you want:

In [11]:
df.mean(axis='columns')

0    0.589594
1    0.486235
2    0.700646
3    0.745043
4    0.284138
dtype: float64

Often times, we want to see all of the common aggregates at once. Pandas provides a function `.describe()` to do this for us:

In [12]:
data.describe()

,year,unemployment_rate,qty_unemployed,qty_employed,labour_force
count,10608.00000,10608.000000,1.060800e+04,1.060800e+04,1.060800e+04
mean,2008.00000,5.943194,1.849809e+05,2.751157e+06,2.936138e+06
std,4.89921,2.261388,2.433025e+05,3.021023e+06,3.242657e+06
min,2000.00000,1.700000,6.697000e+03,2.486420e+05,2.619910e+05
25%,2004.00000,4.400000,4.088675e+04,7.359608e+05,7.836765e+05
50%,2008.00000,5.500000,1.150930e+05,1.837300e+06,1.961558e+06
75%,2012.00000,7.000000,2.212182e+05,3.226836e+06,3.451084e+06
max,2016.00000,17.300000,2.316525e+06,1.819380e+07,1.920923e+07


Some useful aggregation functions in pandas are in the table below:

[insert table]

Whilst these aggregations can provide some notions of patterns in the data - often we want to go deeper - and compute these summaries at the level of a group (like a state of a year). Pandas has the `groupby` function that will allow us to efficiently looks at aggregate statistics on subsets of the data.

## GroupBy: The Split, Apply, Combine Paradigm
The canonical example of the groupby paradigm  is best represented in a figure:

[insert figure]

In words, GroupBy does the following:
* The *split* step breaks up a DataFrame into subsets depending on the value of a specified key
* the *apply* step computes some aggregate functions within an individual group
* The *combine* step merges the results of the computations within groups back to a single dataset

We could certainly perform all these operations by ourselves, manually - but the tedium of the task combined with a high probability of error makes us immediately want to use a built in function that abstracts away from individual steps and asks the user to rather think of the operation as a whole rather than the individual steps.

As a start - let's look at the average unemployment rate by state. As a first step we must specify the name of the column we are looking to split on:

In [13]:
data.groupby('state')

Think of this object as an under the hood view of a `DataFrame`. Note that it does not compute anything until an aggregation is applied to it; i.e. it uses 'lazy evaluation'

Now lets get those mena unemployment rates:

In [15]:
data.groupby('state').unemployment_rate.mean()

state
Alabama                  6.514706
Alaska                   7.005392
Arizona                  6.381373
Arkansas                 5.997059
California               7.443627
Colorado                 5.453922
Connecticut              5.821078
Delaware                 5.214216
District of Columbia     7.306373
Florida                  6.158824
Georgia                  6.446078
Hawaii                   4.456373
Idaho                    5.525000
Illinois                 6.936275
Indiana                  6.184804
Iowa                     4.373039
Kansas                   5.084314
Kentucky                 6.639216
Louisiana                6.197549
Maine                    5.470098
Maryland                 5.121078
Massachusetts            5.521078
Michigan                 7.662255
Minnesota                4.886765
Mississippi              7.285784
Missouri                 6.069118
Montana                  5.023039
Nebraska                 3.629412
Nevada                   7.220098
New Hamp

Notice if we wanted the aggregation function to be applied to all of the columns, we can skip the part where we specify the column:

In [16]:
data.groupby('state').mean()

,year,unemployment_rate,qty_unemployed,qty_employed,labour_force
state,,,,,
Alabama,2008.0,6.514706,1.407627e+05,2.016128e+06,2.156891e+06
Alaska,2008.0,7.005392,2.449233e+04,3.251195e+05,3.496119e+05
Arizona,2008.0,6.381373,1.901428e+05,2.759190e+06,2.949333e+06
Arkansas,2008.0,5.997059,7.955364e+04,1.244863e+06,1.324416e+06
California,2008.0,7.443627,1.347057e+06,1.665503e+07,1.800209e+07
Colorado,2008.0,5.453922,1.452646e+05,2.503614e+06,2.648878e+06
Connecticut,2008.0,5.821078,1.082514e+05,1.736436e+06,1.844688e+06
Delaware,2008.0,5.214216,2.292001e+04,4.152155e+05,4.381355e+05
District of Columbia,2008.0,7.306373,2.490122e+04,3.137833e+05,3.386846e+05


and if we want multiple, but not all columns:

In [17]:
data.groupby('state')['unemployment_rate', 'qty_unemployed'].mean()

,unemployment_rate,qty_unemployed
state,,
Alabama,6.514706,1.407627e+05
Alaska,7.005392,2.449233e+04
Arizona,6.381373,1.901428e+05
Arkansas,5.997059,7.955364e+04
California,7.443627,1.347057e+06
Colorado,5.453922,1.452646e+05
Connecticut,5.821078,1.082514e+05
Delaware,5.214216,2.292001e+04
District of Columbia,7.306373,2.490122e+04


Very Cool!

## Challenge

Try and compute statistics where the grouping is described by multiple variables:
1. Compute the average unemployment rate by state-year
2. Compute the average unemployment rate by state-year-month
3. Compute the average unemployment rate by state-year for the Carolinas (Hint: use the .filter() function after the computation)
4. Compute the average unemployment rate by state-year and use `unstack` to view the data in a way that is different from the default output

#### Partial Solution

In [21]:
data.groupby(['state', 'year'])['unemployment_rate'].mean().filter(like='Carolina')

state           year
North Carolina  2000     3.666667
                2001     5.516667
                2002     6.658333
                2003     6.375000
                2004     5.491667
                2005     5.233333
                2006     4.750000
                2007     4.725000
                2008     6.100000
                2009    10.575000
                2010    10.850000
                2011    10.275000
                2012     9.291667
                2013     7.983333
                2014     6.300000
                2015     5.750000
                2016     5.058333
South Carolina  2000     3.758333
                2001     5.216667
                2002     5.808333
                2003     6.908333
                2004     6.825000
                2005     6.725000
                2006     6.433333
                2007     5.666667
                2008     6.791667
                2009    11.258333
                2010    11.158333
                2011    10.

In [23]:
data.groupby(['state', 'year'])['unemployment_rate'].mean().unstack(level=0)

state,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
year,,,,,,,,,,,,,,,,,,,,,
2000,4.583333,6.375000,3.950000,4.258333,4.941667,2.758333,2.391667,3.733333,5.608333,3.683333,...,2.458333,3.866667,4.291667,3.341667,2.808333,2.300000,5.166667,5.483333,3.525000,3.908333
2001,5.125000,6.433333,4.791667,4.958333,5.433333,3.800000,3.133333,3.500000,6.333333,4.641667,...,3.091667,4.550000,4.941667,4.416667,3.333333,3.233333,6.300000,5.008333,4.525000,3.825000
2002,5.900000,7.291667,6.083333,5.483333,6.666667,5.533333,4.333333,4.025000,6.408333,5.633333,...,3.183333,5.183333,6.341667,5.800000,3.991667,4.216667,7.408333,5.916667,5.375000,4.041667
2003,6.016667,7.825000,5.741667,5.933333,6.800000,5.958333,5.391667,4.266667,6.833333,5.208333,...,3.533333,5.633333,6.683333,5.633333,4.266667,4.108333,7.408333,6.050000,5.700000,4.275000
2004,5.683333,7.458333,5.033333,5.708333,6.208333,5.483333,4.966667,3.991667,7.783333,4.633333,...,3.725000,5.333333,5.941667,4.991667,3.675000,3.825000,6.266667,5.325000,5.025000,3.808333
2005,4.491667,6.883333,4.683333,5.250000,5.400000,5.025000,4.883333,4.141667,6.375000,3.691667,...,3.816667,5.550000,5.408333,4.091667,3.475000,3.566667,5.550000,5.108333,4.741667,3.583333
2006,4.033333,6.616667,4.208333,5.158333,4.900000,4.308333,4.333333,3.575000,5.791667,3.225000,...,3.108333,5.200000,4.891667,2.975000,3.683333,3.100000,5.041667,4.941667,4.741667,3.175000
2007,3.983333,6.333333,3.858333,5.308333,5.375000,3.733333,4.475000,3.441667,5.525000,4.008333,...,2.841667,4.675000,4.291667,2.566667,4.008333,3.033333,4.708333,4.583333,4.900000,2.816667
2008,5.658333,6.691667,6.125000,5.466667,7.275000,4.825000,5.683333,4.941667,6.458333,6.283333,...,3.050000,6.575000,4.808333,3.591667,4.683333,3.941667,5.383333,4.358333,4.908333,3.050000


## More useful GroupBy functionality

Often we want more than one aggregate statistic, for example we may want the min, mean and max by State. We can use the `.aggregate()` function to get these:

In [28]:
data.groupby('state')['unemployment_rate'].aggregate([np.min, np.mean, np.max])

,amin,mean,amax
state,,,
Alabama,3.3,6.514706,12.3
Alaska,5.2,7.005392,9.1
Arizona,3.4,6.381373,11.1
Arkansas,3.4,5.997059,9.5
California,4.4,7.443627,12.7
Colorado,2.4,5.453922,9.2
Connecticut,1.7,5.821078,9.8
Delaware,2.9,5.214216,9.4
District of Columbia,4.7,7.306373,10.9


We can dress it up a little too if we like:

In [34]:
data.groupby('state')['unemployment_rate'] \
        .aggregate({'Min. Unemploy.' : np.min,
                    'Mean Unemploy.' : np.mean,
                    'Max. Unemploy.' : np.max})

,Mean Unemploy.,Min. Unemploy.,Max. Unemploy.
state,,,
Alabama,6.514706,3.3,12.3
Alaska,7.005392,5.2,9.1
Arizona,6.381373,3.4,11.1
Arkansas,5.997059,3.4,9.5
California,7.443627,4.4,12.7
Colorado,5.453922,2.4,9.2
Connecticut,5.821078,1.7,9.8
Delaware,5.214216,2.9,9.4
District of Columbia,7.306373,4.7,10.9


In the challenge above, we used filter to select out states based on a string, but there are more uses of filter than this simple one. For example, we can filter based on the group properties:

In [47]:
def filter_func(x):
    return x['unemployment_rate'].min() > 5

problem_states = data.groupby('state').filter(filter_func)

In [48]:
problem_states.state.unique()

array(['Alaska', 'Puerto Rico'], dtype=object)

In [64]:
problem_states.groupby(['state', 'year']).unemployment_rate.mean().unstack(level=0)

state,Alaska,Puerto Rico
year,,
2000,6.375000,10.125000
2001,6.433333,11.350000
2002,7.291667,12.250000
2003,7.825000,11.966667
2004,7.458333,10.575000
2005,6.883333,11.325000
2006,6.616667,10.500000
2007,6.333333,11.183333
2008,6.691667,11.766667


### Other useful functions:

GroupBy also works alongside the following functions:
* `transform()`
* `apply()`

For lack of time we leave these for you to explore.